In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

def prepare_monetary_sentiment_dataset():
    """
    Create training dataset with monetary policy labels
    Label: 
    - 0 = dovish (expansionary/low inflation risk)
    - 1 = neutral
    - 2 = hawkish (contractionary/high inflation risk)
    """
    
    data = {
        'text': [
            "Energy prices are expected to rise",
            "Unemployment rate decreased to 3.5%",
            "Core inflation remains stable at target",
            "Wage growth accelerates beyond expectations",
            "Economic growth slows to 1.2%",
            "Oil prices decline sharply",
            "Central bank maintains rates unchanged"
        ],
        'label': [
            2,  # Hawkish - inflation pressure
            2,  # Hawkish - tight labor market
            1,  # Neutral - stable inflation
            2,  # Hawkish - wage-price spiral risk
            0,  # Dovish - weak growth
            0,  # Dovish - disinflationary
            1   # Neutral - no change
        ]
    }
    
    return Dataset.from_pandas(pd.DataFrame(data))

def fine_tune_monetary_finbert(train_dataset, output_dir='./monetary-finbert'):
    """
    Fine-tune FinBERT for monetary policy sentiment
    """
    
    # Load pre-trained FinBERT
    model_name = 'ProsusAI/finbert'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(
        model_name, 
        num_labels=3,  # dovish, neutral, hawkish
        ignore_mismatched_sizes=True
    )
    
    # Tokenize dataset
    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=128
        )
    
    tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        learning_rate=2e-5
    )
    
    # Train
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )
    
    trainer.train()
    
    # Save model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return model, tokenizer

# Usage
# dataset = prepare_monetary_sentiment_dataset()
# model, tokenizer = fine_tune_monetary_finbert(dataset)


In [ ]:
data =   